# Benchmarking cell2location pyro model using softplus/exp for scales

In [1]:
import sys, ast, os
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt 
import matplotlib as mpl
data_type = 'float32'

#sys.path.insert(1, '/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/')
#sys.path.insert(1, '/nfs/team205/vk7/sanger_projects/cell2location_dev/')

import cell2location_model 

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
import seaborn as sns

`THEANO_FLAGS='force_device=True'` forces the package to use GPU. Pay attention to error messages that might indicate theano failed to initalise the GPU.   
Do not forget to change `device=cuda4` to your available GPU id. Use `device=cuda` / `device=cuda0` if you have just one locally or if you are requesting one GPU via HPC cluster job.

### The purpose of the notebook is to benchmark several versions of the model using mouse brain data.

In [2]:
sc_data_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_data/mouse_visium_snrna/'
sp_data_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/data/'
results_folder = '/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/'

## Read datasets and train cell2location

In [3]:
adata_vis = anndata.read(f'{sp_data_folder}synth_adata_real_mg_20210131.h5ad')
adata_vis.uns['spatial'] = {'x': 'y'}
adata_vis = adata_vis[adata_vis.obs['sample'] == 'exper0',:]

adata_snrna_raw = anndata.read(f'{sp_data_folder}training_5705STDY8058280_5705STDY8058281_20210131.h5ad')

In [4]:
import scipy
adata_snrna_raw.X = scipy.sparse.csr_matrix(adata_snrna_raw.X)

In [5]:
adata_vis.X = scipy.sparse.csr_matrix(adata_vis.X)

Add counts matrix as `adata.raw` 

In [6]:
adata_snrna_raw.raw = adata_snrna_raw
adata_vis.raw = adata_vis

In [7]:
# compute average for each cluster
aver = cell2location_model.get_cluster_averages(adata_snrna_raw, 'annotation_1')

# make sure the order of gene matches between aver and x_data
aver = aver.loc[adata_vis.var_names,:]

# generate one-hot encoded matrix telling which obs belong to whic samples
obs2sample_df = pd.get_dummies(adata_vis.obs['sample'])

## Model training

In [8]:
results_folder

'/nfs/team205/vk7/sanger_projects/cell2location_paper/notebooks/selected_results/benchmarking/with_tissue_zones/real_mg/pyro/'

In [9]:
import torch
torch.set_default_tensor_type(torch.cuda.FloatTensor)

mod = cell2location_model.LocationModelLinearDependentWMultiExperiment(
    n_obs=adata_vis.n_obs,
    n_var=adata_vis.n_vars,
        n_fact=aver.shape[1],
        n_exper=obs2sample_df.shape[1],
        batch_size=None,
        cell_state_mat=aver.values.astype('float32'),
    )

from pyro.distributions import constraints
mod.guide.scale_constraint = constraints.positive
#mod.guide.scale_constraint = constraints.softplus_positive

#mod.cuda()
mod.train_full_data(x_data=adata_vis.raw.X.toarray().astype('float32'),
                    obs2sample=obs2sample_df.values.astype('float32'),
                    n_epochs=30000, lr=0.002)

/nfs/users/nfs_v/vk7/.local/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:141: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if len(shape) >= -dim and shape[dim] != 1:
/nfs/users/nfs_v/vk7/.local/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:151: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.subsample_size < self.size:
/nfs/users/nfs_v/vk7/.local/lib/python3.7/site-packages/pyro/poutine/subsample_messenger.py:141: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We

In [10]:
mod.model.cell_number_prior["N_cells_per_location"]

8

In [ ]:
rcParams['figure.figsize'] = 5, 5
for m in models_list:
    
    r = cell2location.run_cell2location(
                          sc_data=inf_aver,
                          sp_data=adata_vis[adata_vis.obs['sample'] == 'exper0',:],
        
                          model_name=m,
        
                          verbose=True, return_all=True,

                          summ_sc_data_args={'cluster_col': "annotation_1"},

                          train_args={'n_iter': 30000, 'learning_rate': 0.005,
                                      'total_grad_norm_constraint': 200,
                                      'sample_prior': False, 'n_prior_samples': 20,
                                      'readable_var_name_col': None,
                                      'sample_name_col': 'sample'},

                          posterior_args={'n_samples': 1000},
                          export_args={'path': results_folder,
                                       'save_model': False, 
                                       'run_name_suffix': '_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
                                       'scanpy_coords_name': 'X_spatial'},
                          model_kwargs={
                                        'cell_number_prior': {'cells_per_spot': 9, 'factors_per_spot': 5, 
                                                              'combs_per_spot': 4
                                                             }
                          }
    )

In [ ]:
rcParams['figure.figsize'] = 5, 5
r = cell2location.run_cell2location(
                          sc_data=inf_aver,
                          sp_data=adata_vis[adata_vis.obs['sample'] == 'exper0',:],
        
                          model_name=models_list[0],
        
                          verbose=True, return_all=True,

                          summ_sc_data_args={'cluster_col': "annotation_1"},

                          train_args={'n_iter': 30000, 'learning_rate': 0.005,
                                      'total_grad_norm_constraint': 200,
                                      'sample_prior': False, 'n_prior_samples': 20,
                                      'readable_var_name_col': None,
                                      'sample_name_col': 'sample'},

                          posterior_args={'n_samples': 1000},
                          export_args={'path': results_folder,
                                       'save_model': False, 
                                       'run_name_suffix': '_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
                                       'scanpy_coords_name': 'X_spatial'},
                          model_kwargs={
                                        'cell_number_prior': {'cells_per_spot': 9, 'factors_per_spot': 5, 
                                                              'combs_per_spot': 4
                                                             }
                          }
    )

## Evaluate inferred cell density (`spot_factors`) and mRNA count (`nUMI_factors`)

In [ ]:
r_load = {'run_name': 'LocationModelLinearDependentWMultiExperiment_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2'}
#r_load = {'run_name': 'LocationModel_1experiments_46clusters_2000locations_12281genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2'}

fig_path=results_folder +r_load['run_name']+'/plots'

sp_data_file = results_folder +r_load['run_name']+'/sp.h5ad'
adata_vis_res = anndata.read(sp_data_file)

In [ ]:
file = results_folder + r_load['run_name']+'/model_.p'
r = pickle.load(file = open(file, "rb"))

In [ ]:
# look at NBoverdispersion hyperprior
adata_vis_res.uns['mod']['post_sample_means']['phi_hyp']

In [ ]:
fig_path

In [ ]:
from re import sub
cell_count = adata_vis_res.obs.loc[:, ['cell_abundances_' in i for i in adata_vis_res.obs.columns]]
cell_count.columns =  [sub('cell_abundances_', '', i) for i in cell_count.columns]
cell_count_columns = cell_count.columns
umi_count = adata_vis_res.obs.loc[:, ['UMI_count_' in i for i in adata_vis_res.obs.columns]]
umi_count.columns =  [sub('UMI_count_', '', i) for i in umi_count.columns]

spot_factors = adata_vis_res.obs[['mean_spot_factors' + i for i in cell_count_columns]]
spot_factors.columns =  [sub('mean_spot_factors', '', i) for i in spot_factors.columns]

spot_factors_sd = adata_vis_res.obs[['sd_spot_factors' + i for i in cell_count_columns]]
spot_factors_sd.columns =  [sub('sd_spot_factors', '', i) for i in spot_factors_sd.columns]

infer_cell_count = spot_factors[cell_count.columns]
#infer_cell_count = cells_per_factor[cell_count.columns]

nUMI_factors = adata_vis_res.obs[['mean_nUMI_factors' + i for i in cell_count_columns]]
nUMI_factors.columns =  [sub('mean_nUMI_factors', '', i) for i in nUMI_factors.columns]

In [ ]:
#spot_factors = spot_factors / spot_factors_sd

infer_cell_count = spot_factors[cell_count.columns]
infer_nUMI_count = nUMI_factors[cell_count.columns]

In [ ]:
# mean number of cell types per location
(cell_count.values > 1).sum(1).mean(), (cell_count.values).sum(1).mean(),\
 (infer_cell_count.values > 1).sum(1).mean(), (infer_cell_count.values).sum(1).mean()

In [ ]:
rcParams['figure.figsize'] = 4, 4.5
# look at the numberof cells per spot factors
plt.hist(adata_vis_res.uns['mod']['post_sample_means']['cells_per_spot']);

In [ ]:
# look at the numberof expressed factors
plt.hist(spot_factors.sum(1));

In [ ]:
# look at the number of expressed factors
plt.hist(adata_vis_res.uns['mod']['post_sample_means']['factors_per_combs']);

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_count.values.flatten(), 
           infer_cell_count.values.flatten(),# / np.mean(adata_vis_res.var['gene_level'].values),
           bins=[50, 50], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell count');
plt.ylabel('Inferred cell density');
plt.title(r'Cell count, $R^2$: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           infer_cell_count.values.flatten()), 3)[0,1]));
#plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
plt.savefig(fig_path + '/Cell_density_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 4, 4.5
rcParams["axes.facecolor"] = "white"

plt.hist2d(umi_count.values.flatten(), 
           nUMI_factors.values.flatten() \
           / np.mean(adata_vis_res.var['gene_level'].values),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated mRNA (UMI) count');
plt.ylabel('Inferred UMI count');
plt.title('mRNA count, $R^2$: ' \
          + str(np.round(np.corrcoef(umi_count.values.flatten(), 
           nUMI_factors.values.flatten()), 3)[0,1]));
plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
plt.savefig(fig_path + '/nUMI_count_cor.pdf')

In [ ]:
cell_proportions = (cell_count.T / cell_count.sum(1)).T
cell_proportions.iloc[np.isnan(cell_proportions.values)] = 0
infer_cell_proportions = (infer_cell_count.T / infer_cell_count.sum(1)).T

rcParams['figure.figsize'] = 4, 4
plt.hist2d(cell_proportions.values.flatten(), 
           infer_cell_proportions.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportion');
plt.ylabel('Inferred cell proportion');
plt.title('Cell proportion, $R^2$: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           infer_cell_proportions.values.flatten()), 3)[0,1]));
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.tight_layout()
plt.savefig(fig_path + '/Cell_proportion_cor.pdf')

In [ ]:
umi_count_proportions = (umi_count.T / umi_count.sum(1)).T
umi_count_proportions.iloc[np.isnan(umi_count_proportions.values)] = 0
infer_nUMI_factors = (nUMI_factors.T / nUMI_factors.sum(1)).T

rcParams['figure.figsize'] = 4, 4
plt.hist2d(umi_count_proportions.values.flatten(), 
           infer_nUMI_factors.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated mRNA proportion');
plt.ylabel('Inferred mRNA proportion');
plt.title('Cell proportion, $R^2$: ' \
          + str(np.round(np.corrcoef(umi_count_proportions.values.flatten(), 
           infer_nUMI_factors.values.flatten()), 3)[0,1]));
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.tight_layout()
plt.savefig(fig_path + '/mRNA_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 4, 4
plt.hist2d(cell_count.values.flatten(), 
           infer_nUMI_count.values.flatten(),
           bins=35, norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell count');
plt.ylabel('Inferred mRNA count');
plt.title('mRNA count, cor: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           infer_nUMI_count.values.flatten()), 3)[0,1]));
plt.tight_layout();
#plt.savefig(fig_path + '/mRNA_count_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 4, 4
plt.hist2d(cell_count.values.flatten(), 
           nUMI_factors.values.flatten(),
           bins=35, norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell count');
plt.ylabel('Simulated mRNA count');
plt.title('mRNA count, cor: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           infer_nUMI_count.values.flatten()), 3)[0,1]));
plt.tight_layout();
#plt.savefig(fig_path + '/mRNA_count_cor.pdf')

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
from scipy import interpolate

with plt.style.context('seaborn'):
    seaborn_colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']


def compute_precision_recall(pos_cell_count, infer_cell_proportions, mode='macro'):
    r""" Plot precision-recall curves on average and for each cell type.
    :param pos_cell_count: binary matrix showing which cell types are present in which locations
    :param infer_cell_proportions: inferred locations (the higher the more cells)
    """
    
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import average_precision_score
    
    ### calculating ###
    predictor = infer_cell_proportions.values + np.random.gamma(20, 1e-12, 
                                                                infer_cell_proportions.shape)
    # For each cell type
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i, c in enumerate(infer_cell_proportions.columns):
        precision[c], recall[c], _ = precision_recall_curve(pos_cell_count[:, i],
                                                            predictor[:, i])
        average_precision[c] = average_precision_score(pos_cell_count[:, i], predictor[:, i], average=mode)

    average_precision["averaged"] = average_precision_score(pos_cell_count, predictor,
                                                     average=mode)

    # A "micro-average": quantifying score on all classes jointly
    if mode == 'micro':
        precision_, recall_, threshold = precision_recall_curve(pos_cell_count.ravel(),
                                                                        predictor.ravel())
        #precision_[threshold < 0.1] = 0
        precision["averaged"], recall["averaged"] =  precision_, recall_

    elif mode == 'macro':
        precisions = []
        recall_grid = np.linspace(0, 1, 2000)

        for i, c in enumerate(infer_cell_proportions.columns):
            f = interpolate.interp1d(recall[c], precision[c])
            precision_interp = f(recall_grid)
            precisions.append(precision_interp)

        precision["averaged"] = np.mean(precisions, axis=0)
        recall['averaged'] = recall_grid

        
    return precision, recall, average_precision

def compute_roc(pos_cell_count, infer_cell_proportions, mode='macro'):
    r""" Plot ROC curves on average and for each cell type.
    :param pos_cell_count: binary matrix showing which cell types are present in which locations
    :param infer_cell_proportions: inferred locations (the higher the more cells)
    """
    
    from sklearn.metrics import roc_curve
    from sklearn.metrics import roc_auc_score
    
    ### calculating ###
    predictor = infer_cell_proportions.values + np.random.gamma(20, 1e-12, 
                                                                infer_cell_proportions.shape)
    # For each cell type
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i, c in enumerate(infer_cell_proportions.columns):
        precision[c], recall[c], _ = roc_curve(pos_cell_count[:, i],
                                                            predictor[:, i])
        average_precision[c] = roc_auc_score(pos_cell_count[:, i], predictor[:, i], average=mode)

    average_precision["averaged"] = roc_auc_score(pos_cell_count, predictor,
                                                     average=mode)

    # A "micro-average": quantifying score on all classes jointly
    if mode == 'micro':
        precision_, recall_, threshold = roc_curve(pos_cell_count.ravel(),
                                                                        predictor.ravel())
        #precision_[threshold < 0.1] = 0
        precision["averaged"], recall["averaged"] =  precision_, recall_

    elif mode == 'macro':
        precisions = []
        recall_grid = np.linspace(0, 1, 2000)

        for i, c in enumerate(infer_cell_proportions.columns):
            f = interpolate.interp1d(recall[c], precision[c])
            precision_interp = f(recall_grid)
            precisions.append(precision_interp)

        precision["averaged"] = np.mean(precisions, axis=0)
        recall['averaged'] = recall_grid

        
    return precision, recall, average_precision

def plot_precision_recall(pos_cell_count, infer_cell_proportions,
                          title='', iso_f1_curves=False,
                          individual_cell_types=True,
                          save=None, mode='macro', curve='PR'):
    r""" Plot precision-recall curves on average and for each cell type.
    :param pos_cell_count: binary matrix showing which cell types are present in which locations
    :param infer_cell_proportions: inferred locations (the higher the more cells)
    :param title: plot title
    :param iso_f1_curves: plot iso curves for F1 score?
    :param individual_cell_types: show precision-recall curves for each cell type?
    """
    
    ### calculating ###
    if curve == 'PR':
        precision, recall, average_precision = compute_precision_recall(pos_cell_count, 
                                                                        infer_cell_proportions, 
                                                                        mode=mode)
        xlabel = 'Recall'
        ylabel = 'Precision'
        
    elif curve == 'ROC':
        recall, precision, average_precision = compute_roc(pos_cell_count, 
                                                                        infer_cell_proportions, 
                                                                        mode=mode)
        xlabel = 'FPR'
        ylabel = 'TPR'
        
    ### plotting ###
    from itertools import cycle
    # setup plot details
    colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

    lines = []
    labels = []
    
    if iso_f1_curves:
        f_scores = np.linspace(0.2, 0.8, num=4)
        
        for f_score in f_scores:
            x = np.linspace(0.01, 1)
            y = f_score * x / (2 * x - f_score)
            l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
            plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

        lines.append(l)
        labels.append('iso-f1 curves')
    
    l, = plt.plot(recall["averaged"], precision["averaged"], color='navy', lw=2)
    lines.append(l)
    labels.append(f'{mode}-average {curve} ({curve} score = {round(average_precision["averaged"], 2)})')

    if individual_cell_types:
        for i, color in zip(infer_cell_proportions.columns, colors):
            l, = plt.plot(recall[i], precision[i], color='gold', lw=1)
            lines.append(l)
        labels.append(f'{curve} for individual cell types')



    fig = plt.gcf()
    fig.subplots_adjust(bottom=0.25)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend(lines, labels, loc=(0, -.37), prop=dict(size=8))
    
    if save is not None:
        plt.savefig(save)

def compare_precision_recall(pos_cell_count, infer_cell_proportions,
                             method_title, title='',
                             legend_loc=(0, -.37),
                             colors=sc.pl.palettes.default_102,
                             mode='macro', curve='PR'):
    
    r""" Plot precision-recall curves on average and for each cell type.
    :param pos_cell_count: binary matrix showing which cell types are present in which locations
    :param infer_cell_proportions: inferred locations (the higher the more cells), 
                                        list of inferred parameters for several methods
    :param method_title: title for each infer_cell_proportions
    :param title: plot title
    """
    # setup plot details
    from itertools import cycle
    colors = cycle(colors)
    lines = []
    labels = []
    roc = {}
    
    ### plotting ###
    for i, color in zip(range(len(infer_cell_proportions)), colors):
        
        if curve == 'PR':
            precision, recall, average_precision = compute_precision_recall(pos_cell_count, 
                                                                        infer_cell_proportions[i], 
                                                                        mode=mode)

            xlabel = 'Recall'
            ylabel = 'Precision'
            
            l, = plt.plot(recall["averaged"], precision["averaged"], color=color, lw=3)

        elif curve == 'ROC':
            FPR, TPR, average_precision = compute_roc(pos_cell_count, 
                                                                        infer_cell_proportions[i], 
                                                                        mode=mode)

            xlabel = 'FPR'
            ylabel = 'TPR'
            
            l, = plt.plot(FPR["averaged"], TPR["averaged"], color=color, lw=3)
            
        
        lines.append(l)

        labels.append(method_title[i] + '(' + curve + ' score = {0:0.2f})'
                      ''.format(average_precision["averaged"]))
        roc[method_title[i]] = average_precision["averaged"]


    fig = plt.gcf()
    fig.subplots_adjust(bottom=0.25)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    if legend_loc is not None:
        plt.legend(lines, labels, loc=legend_loc, prop=dict(size=14))

    #plt.show()
    return roc


In [ ]:
rcParams['figure.figsize'] = 7, 32
rcParams['pdf.fonttype'] = 42
pos_cell_count = cell_count.values > 0.1

def pr_by_category(pos_cell_count, infer_cell_count, design,
                   nrow=5, ncol=1, title='', fig_path=''):

    design_loc = design.loc[cell_count.columns,:]
    
    plt.subplot(nrow, ncol, 1)
    ind = (design_loc['is_uniform'] * design_loc['is_high_density']).values.astype(bool)
    plot_precision_recall(pos_cell_count[:, ind], infer_cell_count.loc[:,ind],
                          title=f'{title} Spatially uniform & high abundance ({ind.sum()})',
                          #save=fig_path + '/Uniform_low_abundance.pdf'
                         )
    
    plt.subplot(nrow, ncol, 2)
    ind = (design_loc['is_uniform'] * (1 - design_loc['is_high_density'])).values.astype(bool)
    plot_precision_recall(pos_cell_count[:, ind], infer_cell_count.loc[:,ind],
                          title=f'{title} Spatially uniform & low abundance ({ind.sum()})',
                          #save=fig_path + '/Uniform_high_abundance.pdf'
                         )
    
    plt.subplot(nrow, ncol, 3)
    ind = ((1 - design_loc['is_uniform']) * design_loc['is_high_density']).values.astype(bool)
    plot_precision_recall(pos_cell_count[:, ind], infer_cell_count.loc[:,ind],
                          title=f'{title} Sparse & high abundance ({ind.sum()})',
                          #save=fig_path + '/Segregated_low_abundance.pdf'
                         )
    
    plt.subplot(nrow, ncol, 4)
    ind = ((1 - design_loc['is_uniform']) * (1 - design_loc['is_high_density'])).values.astype(bool)
    plot_precision_recall(pos_cell_count[:, ind], infer_cell_count.loc[:,ind],
                          title=f'{title} Sparse & low abundance ({ind.sum()})',
                          #save=fig_path + '/Segregated_high_abundance.pdf'
                         )
    
    plt.subplot(nrow, ncol, 5)
    plot_precision_recall(pos_cell_count, infer_cell_count,
                          title=title + ' All',
                          #save=fig_path + '/All.pdf'
                         )
    
from re import sub
model_name = sub('_.+$', '', r['run_name'])
pr_by_category(pos_cell_count, infer_cell_count, adata_vis_res.uns['design']['cell_types2zones'],
               nrow=5, ncol=1, title=model_name,#r['model_name'],#.__name__,
               fig_path=fig_path)
plt.tight_layout()
plt.savefig(fig_path + '/All.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 32
rcParams['pdf.fonttype'] = 42
pos_cell_count = cell_count.values > 0.1

pr_by_category(pos_cell_count, infer_cell_proportions, adata_vis_res.uns['design']['cell_types2zones'],
               nrow=5, ncol=1, title=model_name,# r['model_name'],#.__name__,
               fig_path=fig_path)
plt.tight_layout()

#### Split results by spatial distribution and abundance of cell types

In [ ]:
def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)

    return np.sum(np.where(a != 0, a * np.log2(a / b), 0))
from scipy.spatial.distance import jensenshannon

def hist_obs_sim(cell_count, infer_cell_count,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 title='', compute_kl=True, equal=True, max_val=1):
    
    cor = np.round(np.corrcoef(cell_count.values.flatten(), 
               infer_cell_count.values.flatten()), 3)[0,1]
    title = title +'\n'+ r'$R^2$: ' + str(cor)
    
    if compute_kl:
        
        #kl = [KL(cell_count.values[r,:], infer_cell_count.values[r,:]) 
        #         for r in range(cell_count.shape[0])]
        js = np.array([jensenshannon(cell_count.values[r,:], infer_cell_count.values[r,:]) 
                 for r in range(cell_count.shape[0])])
        #kl = np.mean(kl)
        js = np.mean(js[~np.isnan(js)])
        #title = title + '\nAverage KL: ' + str(np.round(kl, 2))
        title = title + '\nAverage JSD: ' + str(np.round(js, 2))
    
    plt.hist2d(cell_count.values.flatten(), 
               infer_cell_count.values.flatten(),
               bins=[35, 35], norm=matplotlib.colors.LogNorm());
    plt.xlabel(xlab);
    plt.ylabel(ylab);
    if equal:
        plt.gca().set_aspect('equal', adjustable='box')
    plt.xlim(0, max_val);
    plt.ylim(0, max_val);
    plt.title(title);
    
def hist_by_category(cell_count, infer_cell_count, design,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 nrow=1, ncol=4, compute_kl=True, equal=True):

    design_loc = design.loc[cell_count.columns,:]
    
    max_val = np.array([cell_count.values.max(), infer_cell_count.values.max()]).max()
    if max_val < 1:
        max_val = 1
    
    plt.subplot(nrow, ncol, 1)
    ind = (design_loc['is_uniform'] * design_loc['is_high_density']).values.astype(bool)
    hist_obs_sim(cell_count.loc[:,ind], infer_cell_count.loc[:,ind],
                 xlab=xlab,
                 ylab=ylab,
                 title=f'Uniform & high abundance ({ind.sum()})',
                 compute_kl=compute_kl, equal=equal, max_val=max_val)
    
    plt.subplot(nrow, ncol, 2)
    ind = (design_loc['is_uniform'] * (1 - design_loc['is_high_density'])).values.astype(bool)
    hist_obs_sim(cell_count.loc[:,ind], infer_cell_count.loc[:,ind],
                 xlab=xlab,
                 ylab=ylab,
                 title=f'Uniform & low abundance ({ind.sum()})',
                 compute_kl=compute_kl, equal=equal, max_val=max_val)
    
    plt.subplot(nrow, ncol, 3)
    ind = ((1 - design_loc['is_uniform']) * design_loc['is_high_density']).values.astype(bool)
    hist_obs_sim(cell_count.loc[:,ind], infer_cell_count.loc[:,ind],
                 xlab=xlab,
                 ylab=ylab,
                 title=f'Sparse & high abundance ({ind.sum()})',
                 compute_kl=compute_kl, equal=equal, max_val=max_val)
    
    plt.subplot(nrow, ncol, 4)
    ind = ((1 - design_loc['is_uniform']) * (1 - design_loc['is_high_density'])).values.astype(bool)
    hist_obs_sim(cell_count.loc[:,ind], infer_cell_count.loc[:,ind],
                 xlab=xlab,
                 ylab=ylab,
                 title=f'Sparse & low abundance ({ind.sum()})',
                 compute_kl=compute_kl, equal=equal, max_val=max_val)
    
cell_proportions = (cell_count.T / cell_count.sum(1)).T
cell_proportions.iloc[np.isnan(cell_proportions.values)] = 0
infer_cell_proportions = (infer_cell_count.T / infer_cell_count.sum(1)).T

rcParams['figure.figsize'] = 18,4.5 #9,9 #20, 4.5
rcParams["axes.facecolor"] = "white"
hist_by_category(cell_proportions, infer_cell_proportions, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 #compute_kl=False, 
                 nrow=1, ncol=4, equal=True)
plt.tight_layout()

plt.savefig(fig_path + '/hist_by_category.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
rcParams["axes.facecolor"] = "white"
hist_by_category(cell_count, infer_cell_count, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell count',
                 ylab='Inferred/predicted cell density',
                 compute_kl=False, equal=False)
plt.tight_layout()
plt.savefig(fig_path + '/hist_density_by_category.pdf')

plt.hist(adata_vis_res.uns['mod']['post_sample_means']['factors_per_spot'].flatten());

In [ ]:
plt.hist(adata_vis_res.uns['mod']['post_sample_means']['combs_per_spot'].flatten());

In [ ]:
combs_factors_df = pd.DataFrame(adata_vis_res.uns['mod']['post_sample_means']['combs_factors'],
                            columns=['comb_' + str(i) for i in range(adata_vis_res.uns['mod']['post_sample_means']['combs_factors'].shape[1])])
combs_factors_df

In [ ]:
from cell2location.plt.plot_heatmap import clustermap
comb2fact_df = pd.DataFrame(adata_vis_res.uns['mod']['post_sample_means']['comb2fact'],
                            columns=adata_vis_res.uns['mod']['fact_names'])
comb2fact_df_prop = (comb2fact_df / comb2fact_df.sum(0))
clustermap(comb2fact_df)
clustermap(comb2fact_df_prop)

#### Stratify cell types by their size in the reference

In [ ]:
gene_factors = pd.DataFrame(adata_vis_res.uns['mod']['post_sample_means']['gene_factors'],
                            index=adata_vis_res.uns['mod']['var_names'],
                            columns=adata_vis_res.uns['mod']['fact_names'])

In [ ]:
plt.hist(gene_factors.sum(0), bins=30);
plt.vlines(2500, 0, 6);
small_ind = np.array(gene_factors.sum(0) < 2500) \
& (adata_vis_res.uns['design']['cell_types2zones']['is_high_density']).values.astype(bool)
gene_factors.columns[small_ind]

In [ ]:
def hist_by_category2(cell_count, infer_cell_count, index,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='', name2='',
                 nrow=1, ncol=4, compute_kl=True, equal=True):

    design_loc = adata_vis_res.uns['design']['cell_types2zones'].loc[cell_count.columns,:]
    
    plt.subplot(nrow, ncol, 1)
    hist_obs_sim(cell_count.loc[:,index], infer_cell_count.loc[:,index.astype(bool)],
                 xlab=xlab,
                 ylab=ylab,
                 title=name1,
                 compute_kl=compute_kl, equal=equal)
    
    plt.subplot(nrow, ncol, 2)
    hist_obs_sim(cell_count.loc[:,~index], infer_cell_count.loc[:,~index.astype(bool)],
                 xlab=xlab,
                 ylab=ylab,
                 title=name2,
                 compute_kl=compute_kl, equal=equal)
    
rcParams['figure.figsize'] = 9, 4.5
rcParams["axes.facecolor"] = "white"

hist_by_category2(cell_proportions, infer_cell_proportions, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

In [ ]:
hist_by_category2(cell_count, infer_cell_count, small_ind,
                 xlab='Simulated cell count',
                 ylab='Inferred cell density',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=False, equal=False)

### Evaluate inferred gene expression level difference between technologies (`gene_level`)

In [ ]:
gene_level = pd.DataFrame(adata_vis_res.uns['mod']['post_sample_means']['gene_level'].flatten(),
                 index=adata_vis_res.uns['mod']['var_names'],
                 columns=['inferred_gene_level'])
adata_vis_res.var['inferred_gene_level'] = gene_level.reindex(index=adata_vis_res.var_names)['inferred_gene_level']

In [ ]:
rcParams['figure.figsize'] = 4, 4.5
var = adata_vis_res.var
var = var.loc[~var['inferred_gene_level'].isna(),:]
plt.hist2d(var['gene_level'].values.flatten(), 
           var['inferred_gene_level'].values.flatten(),
           bins=35, norm=matplotlib.colors.LogNorm());
plt.xlabel(r'Simulated gene level $M_g$');
plt.ylabel('Inferred gene level $M_g$');
#plt.xlim(0, 1);
#plt.ylim(0, 1);
plt.title('Gene level, cor: ' \
          + str(np.round(np.corrcoef(var['gene_level'].values.flatten(), 
                var['inferred_gene_level'].values.flatten()), 3)[0,1]));
plt.tight_layout();
plt.savefig(fig_path + '/hist_gene_level.pdf')

### Evaluate inferred cell density inferred with stereoscope

In [ ]:
stereo_path_1 = f'{results_folder}scvi_stereoscope/'
scvi_stereoscope = anndata.read(f'{stereo_path_1}sp_n_epochs30k.h5ad')

scvi_stereoscope = scvi_stereoscope.obsm['decomposition'].copy()
scvi_stereoscope = (scvi_stereoscope.T / scvi_stereoscope.sum(1)).T
scvi_stereoscope = scvi_stereoscope.loc[cell_count.index,:]

In [ ]:
stereo_df = scvi_stereoscope[cell_count.columns]

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_proportions.values.flatten(), 
           stereo_df.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportions');
plt.ylabel('Inferred cell proportions');
plt.title('Cell proportions, cor: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           stereo_df.values.flatten()), 3)[0,1]));
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.tight_layout();

plt.savefig(os.path.split(stereo_path_1)[0] + '/Cell_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
hist_by_category(cell_proportions, stereo_df, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion')
plt.tight_layout()

plt.savefig(os.path.split(stereo_path_1)[0] + '/Cell_proportion_cor_by_group.pdf')

In [ ]:
rcParams['figure.figsize'] = 9, 4.5
hist_by_category2(cell_proportions, stereo_df, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

plt.savefig(os.path.split(stereo_path_1)[0] + '/Cell_proportion_cor_by_size.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 40
pos_cell_count = cell_count.values > 0.1
pr_by_category(pos_cell_count, stereo_df, adata_vis_res.uns['design']['cell_types2zones'],
                 nrow=5, ncol=1,
                      title='Stereoscope')

plt.savefig(os.path.split(stereo_path_1)[0] + '/ROC_all.pdf')

### Evaluate inferred cell density inferred with nnls (autogenes)

In [ ]:
# read cell proportions
nnls_path_1 = results_folder + 'autogenes_models/coef_nnls_seed.csv'
nnls_df = pd.read_csv(nnls_path_1, sep=',', index_col=0).T

# compute proportions per location
nnls_df = (nnls_df.T / nnls_df.sum(1)).T

adata_vis_plt = adata_vis.copy()
nnls_df = nnls_df.loc[nnls_df.index.isin(adata_vis_plt.obs_names)]
adata_vis_plt = adata_vis_plt[nnls_df.index,:]

adata_vis_plt.obs['imagecol'] = adata_vis_plt.obsm['X_spatial'][:,0]
adata_vis_plt.obs['imagerow'] = - adata_vis_plt.obsm['X_spatial'][:,1]

In [ ]:
nnls_df = nnls_df[cell_count.columns]

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_proportions.values.flatten(), 
           nnls_df.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportions');
plt.ylabel('Inferred cell proportions');
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.title('Cell proportions, cor: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           nnls_df.values.flatten()), 3)[0,1]));

plt.savefig(os.path.split(nnls_path_1)[0] + '/Cell_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
hist_by_category(cell_proportions, nnls_df, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion')
plt.tight_layout()

plt.savefig(os.path.split(nnls_path_1)[0] + '/Cell_proportion_cor_by_group.pdf')

In [ ]:
rcParams['figure.figsize'] = 9, 4.5
hist_by_category2(cell_proportions, nnls_df, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

plt.savefig(os.path.split(nnls_path_1)[0] + '/Cell_proportion_cor_by_size.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 40
pos_cell_count = cell_count.values > 0.1
pr_by_category(pos_cell_count, nnls_df, adata_vis_res.uns['design']['cell_types2zones'],
                 nrow=5, ncol=1,
                      title='NNLS')

plt.savefig(os.path.split(nnls_path_1)[0] + '/ROC_all.pdf')

### Evaluate inferred cell density inferred with RCTD

In [ ]:
# read cell proportions
RCTD_path_1 = results_folder + 'rctd/results_weights.csv'
RCTD_df = pd.read_csv(RCTD_path_1, sep=',', index_col='spot')

# compute proportions per location
RCTD_df_orig = RCTD_df.copy()
RCTD_df = (RCTD_df.T / RCTD_df.sum(1)).T

adata_vis_plt = adata_vis.copy()
RCTD_df = RCTD_df.loc[RCTD_df.index.isin(adata_vis_plt.obs_names)]
adata_vis_plt = adata_vis_plt[RCTD_df.index,:]

adata_vis_plt.obs['imagecol'] = adata_vis_plt.obsm['X_spatial'][:,0]
adata_vis_plt.obs['imagerow'] = - adata_vis_plt.obsm['X_spatial'][:,1]

In [ ]:
RCTD_df = RCTD_df[cell_count.columns]
RCTD_df_orig = RCTD_df_orig[cell_count.columns]

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_count.values.flatten(), 
           RCTD_df_orig.values.flatten(),# / np.mean(adata_vis_res.var['gene_level'].values),
           bins=[50, 50], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell count');
plt.ylabel('Inferred cell density');
plt.title(r'Cell count, $R^2$: ' \
          + str(np.round(np.corrcoef(cell_count.values.flatten(), 
           RCTD_df_orig.values.flatten()), 3)[0,1]));
#plt.gca().set_aspect('equal', adjustable='box')
plt.tight_layout()
plt.savefig(os.path.split(RCTD_path_1)[0] + '/RCTD_Cell_density_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_proportions.values.flatten(), 
           RCTD_df.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportions');
plt.ylabel('Inferred cell proportions');
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Cell proportions, cor: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           RCTD_df.values.flatten()), 3)[0,1]));

plt.savefig(os.path.split(RCTD_path_1)[0] + '/Cell_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
hist_by_category(cell_proportions, RCTD_df, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion')
plt.tight_layout()

plt.savefig(os.path.split(RCTD_path_1)[0] + '/Cell_proportion_cor_by_group.pdf')

In [ ]:
rcParams['figure.figsize'] = 9, 4.5
hist_by_category2(cell_proportions, RCTD_df, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

plt.savefig(os.path.split(RCTD_path_1)[0] + '/Cell_proportion_cor_by_size.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 40
pos_cell_count = cell_count.values > 0.1
pr_by_category(pos_cell_count, RCTD_df, adata_vis_res.uns['design']['cell_types2zones'],
                 nrow=5, ncol=1,
                      title='RCTD')

plt.savefig(os.path.split(RCTD_path_1)[0] + '/ROC_all.pdf')

### Evaluate inferred cell propotion inferred with Seurat 3

In [ ]:
# read cell proportions
seurat_path_1 = results_folder + 'seurat/results_pca.csv'
seurat_df = pd.read_csv(seurat_path_1, sep=',', index_col='spot')
seurat_df = seurat_df.drop(columns=['predicted.id', 'prediction.score.max', 'Unnamed: 0'])
from re import sub
seurat_df.columns = [sub('prediction.score.', '', i) for i in seurat_df.columns]

# compute proportions per location
seurat_df = (seurat_df.T / seurat_df.sum(1)).T

adata_vis_plt = adata_vis.copy()
seurat_df = seurat_df.loc[seurat_df.index.isin(adata_vis_plt.obs_names)]
adata_vis_plt = adata_vis_plt[seurat_df.index,:]

adata_vis_plt.obs['imagecol'] = adata_vis_plt.obsm['X_spatial'][:,0]
adata_vis_plt.obs['imagerow'] = - adata_vis_plt.obsm['X_spatial'][:,1]

In [ ]:
seurat_df = seurat_df[cell_count.columns]

In [ ]:
rcParams['figure.figsize'] = 4, 4
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_proportions.values.flatten(), 
           seurat_df.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportions');
plt.ylabel('Inferred cell proportions');
plt.title('Cell proportions, cor: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           seurat_df.values.flatten()), 3)[0,1]));
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.tight_layout();

plt.savefig(os.path.split(seurat_path_1)[0] + '/Cell_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
hist_by_category(cell_proportions, seurat_df, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion')
plt.tight_layout()

plt.savefig(os.path.split(seurat_path_1)[0] + '/Cell_proportion_cor_by_group.pdf')

In [ ]:
rcParams['figure.figsize'] = 9, 4.5
hist_by_category2(cell_proportions, seurat_df, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

plt.savefig(os.path.split(seurat_path_1)[0] + '/Cell_proportion_cor_by_size.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 40
pos_cell_count = cell_count.values > 0.1
pr_by_category(pos_cell_count, seurat_df, adata_vis_res.uns['design']['cell_types2zones'],
                 nrow=5, ncol=1,
                      title='Seurat 3',
               fig_path=fig_path)

plt.savefig(os.path.split(seurat_path_1)[0] + '/ROC_all.pdf')

In [ ]:
# read cell proportions
seurat_path_1 = results_folder + 'seurat/results_cca.csv'
seurat_cca_df = pd.read_csv(seurat_path_1, sep=',', index_col='spot')
seurat_cca_df = seurat_cca_df.drop(columns=['predicted.id', 'prediction.score.max', 'Unnamed: 0'])
from re import sub
seurat_cca_df.columns = [sub('prediction.score.', '', i) for i in seurat_cca_df.columns]

# compute proportions per location
seurat_cca_df = (seurat_cca_df.T / seurat_cca_df.sum(1)).T

adata_vis_plt = adata_vis.copy()
seurat_cca_df = seurat_cca_df.loc[seurat_cca_df.index.isin(adata_vis_plt.obs_names)]
adata_vis_plt = adata_vis_plt[seurat_cca_df.index,:]

adata_vis_plt.obs['imagecol'] = adata_vis_plt.obsm['X_spatial'][:,0]
adata_vis_plt.obs['imagerow'] = - adata_vis_plt.obsm['X_spatial'][:,1]

In [ ]:
seurat_cca_df = seurat_cca_df[cell_count.columns]

In [ ]:
rcParams['figure.figsize'] = 4, 4.5
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_proportions.values.flatten(), 
           seurat_cca_df.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportions');
plt.ylabel('Inferred cell proportions');
plt.title('Cell proportions, cor: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           seurat_cca_df.values.flatten()), 3)[0,1]));
plt.gca().set_aspect('equal', adjustable='box')
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.tight_layout();

plt.savefig(os.path.split(seurat_path_1)[0] + '/cca_Cell_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
hist_by_category(cell_proportions, seurat_cca_df, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion')
plt.tight_layout()

plt.savefig(os.path.split(seurat_path_1)[0] + '/cca_Cell_proportion_cor_by_group.pdf')

In [ ]:
rcParams['figure.figsize'] = 9, 4.5
hist_by_category2(cell_proportions, seurat_cca_df, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

plt.savefig(os.path.split(seurat_path_1)[0] + '/cca_Cell_proportion_cor_by_size.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 40
pos_cell_count = cell_count.values > 0.1
pr_by_category(pos_cell_count, seurat_cca_df, adata_vis_res.uns['design']['cell_types2zones'],
                 nrow=5, ncol=1,
                      title='Seurat 3',
               fig_path=fig_path)

plt.savefig(os.path.split(seurat_path_1)[0] + '/cca_ROC_all.pdf')

### Evaluate inferred cell propotion inferred with SPOTlight

In [ ]:
# read cell proportions
spotlight_path_1 = results_folder + 'SPOTlight/results_hvg5k.csv'
spotlight_df = pd.read_csv(spotlight_path_1, sep=',', index_col='rn')
spotlight_df = spotlight_df.drop(columns=['res_ss'])

from re import sub

seu_index = [sub(' ', '.',i) for i in cell_proportions.columns]
seu_index = [sub('/', '.',i) for i in seu_index]
seu_index = [sub('-', '.',i) for i in seu_index]
seu_index = [sub('_', '.',i) for i in seu_index]

spotlight_df = spotlight_df[seu_index]
spotlight_df.columns = cell_proportions.columns

# compute proportions per location (already proportions)
#spotlight_df = (spotlight_df.T / spotlight_df.sum(1)).T

adata_vis_plt = adata_vis.copy()
spotlight_df = spotlight_df.loc[spotlight_df.index.isin(adata_vis_plt.obs_names)]
adata_vis_plt = adata_vis_plt[spotlight_df.index,:]

adata_vis_plt.obs['imagecol'] = adata_vis_plt.obsm['X_spatial'][:,0]
adata_vis_plt.obs['imagerow'] = - adata_vis_plt.obsm['X_spatial'][:,1]

In [ ]:
spotlight_df = spotlight_df[cell_count.columns]

In [ ]:
rcParams['figure.figsize'] = 4, 4.5
rcParams["axes.facecolor"] = "white"

plt.hist2d(cell_proportions.values.flatten(), 
           spotlight_df.values.flatten(),
           bins=[35, 41], norm=matplotlib.colors.LogNorm());
plt.xlabel('Simulated cell proportions');
plt.ylabel('Inferred cell proportions');
plt.xlim(0, 1);
plt.ylim(0, 1);
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Cell proportions, cor: ' \
          + str(np.round(np.corrcoef(cell_proportions.values.flatten(), 
           spotlight_df.values.flatten()), 3)[0,1]));

plt.savefig(os.path.split(spotlight_path_1)[0] + '/spotlightCell_proportion_cor.pdf')

In [ ]:
rcParams['figure.figsize'] = 18, 4.5
hist_by_category(cell_proportions, spotlight_df, adata_vis_res.uns['design']['cell_types2zones'],
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion')
plt.tight_layout()

plt.savefig(os.path.split(spotlight_path_1)[0] + '/spotlightCell_proportion_cor_by_group.pdf')

In [ ]:
rcParams['figure.figsize'] = 9, 4.5
hist_by_category2(cell_proportions, spotlight_df, small_ind,
                 xlab='Simulated cell proportion',
                 ylab='Inferred cell proportion',
                 name1='Small & low abundance cells', name2='Large cells',
                 nrow=1, ncol=2, compute_kl=True)

plt.savefig(os.path.split(spotlight_path_1)[0] + '/spotlightCell_proportion_cor_by_size.pdf')

In [ ]:
rcParams['figure.figsize'] = 7, 40
pos_cell_count = cell_count.values > 0.1
pr_by_category(pos_cell_count, spotlight_df, adata_vis_res.uns['design']['cell_types2zones'],
                 nrow=5, ncol=1,
                      title='SPOTlight',
               fig_path=fig_path)

plt.savefig(os.path.split(spotlight_path_1)[0] + '/spotlightROC_all.pdf')

## Compare per-spot accuracy of cell2location and stereoscope

### Compare the stereoscope and our model using precision-recall

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt

with plt.style.context('seaborn'):
    seaborn_colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']
seaborn_colors
sc.pl.palettes.default_102
    
def read_cell_maps(results_folder, run_name, cell_count, post_type='mean'):
    
    # read anndata object, extract data
    sp_data_file = results_folder + run_name + '/sp.h5ad'
    adata_vis_res = anndata.read(sp_data_file)
    
    # read pickle with extra info
    file = results_folder + run_name+'/model_.p'
    r = pickle.load(file = open(file, "rb"))
    
    # extract data
    from re import sub
    cell_count_columns = cell_count.columns

    spot_factors = adata_vis_res.obs[[post_type + '_spot_factors' + i for i in cell_count_columns]]
    spot_factors.columns =  [sub(post_type + '_spot_factors', '', i) for i in spot_factors.columns]

    #spot_factors_sd = adata_vis_res.obs[['sd_spot_factors' + i for i in cell_count_columns]]
    #spot_factors_sd.columns =  [sub('sd_spot_factors', '', i) for i in spot_factors_sd.columns]

    nUMI_factors = adata_vis_res.obs[[post_type + '_nUMI_factors' + i for i in cell_count_columns]]
    nUMI_factors.columns =  [sub(post_type + '_nUMI_factors', '', i) for i in nUMI_factors.columns]
    
    #spot_factors = spot_factors / spot_factors_sd

    infer_cell_count = spot_factors[cell_count.columns]
    infer_nUMI_count = nUMI_factors[cell_count.columns]
    
    infer_cell_proportions = (infer_cell_count.T / infer_cell_count.sum(1)).T
    
    return r['model_name'], infer_cell_proportions, infer_cell_count, infer_nUMI_count # spot_factors_sd

In [ ]:
names = []
results = []
names_count = []
results_count = []
results_q05 = []
results_count_q05 = []

run_names = [
    'LocationModelLinearDependentWMultiExperiment_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
    'LocationModelLinearDependentWMultiExperimentNoMg_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
    'LocationModelHierarchicalWMultiExperiment_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
    'LocationModelMultiExperiment_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
    'LocationModelLinearDependentWMultiExperimentNoSegLs_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
    'LocationModelLinearDependentWMultiExperimentLocationBackgroundNormGeneAlpha_1experiments_49clusters_2500locations_12080genes_defaults_ceps9_cps4_fps5_inf_shape01_gl2',
            ]

for n in run_names:
    name, infer_cell_prop, infer_cell_count, infer_nUMI_count = \
    read_cell_maps(results_folder, n, cell_count, post_type='mean')
    #infer_cell_count = (infer_cell_count.T / infer_cell_count.max(1)).T
    names.append(name)
    results.append(infer_cell_prop)
    names_count.append(name)
    results_count.append(infer_cell_count)
    
    name, infer_cell_prop, infer_cell_count, infer_nUMI_count = \
    read_cell_maps(results_folder, n, cell_count, post_type='q05')
    #infer_cell_count = (infer_cell_count.T / infer_cell_count.max(1)).T
    results_q05.append(infer_cell_prop)
    results_count_q05.append(infer_cell_count)

# add stereoscope
results.append(stereo_df[cell_count.columns])
results_count.append(stereo_df[cell_count.columns])
results_q05.append(stereo_df[cell_count.columns])
results_count_q05.append(stereo_df[cell_count.columns])
names.append('Stereoscope')

# add RCTD_df
results.append(RCTD_df[cell_count.columns])
results_count.append(RCTD_df[cell_count.columns])
results_q05.append(RCTD_df[cell_count.columns])
results_count_q05.append(RCTD_df[cell_count.columns])
names.append('RCTD')

# add Seurat 3
results.append(seurat_df[cell_count.columns])
results_count.append(seurat_df[cell_count.columns])
results_q05.append(seurat_df[cell_count.columns])
results_count_q05.append(seurat_df[cell_count.columns])
names.append('Seurat 3 PCA')

# add Seurat 3
results.append(seurat_cca_df[cell_count.columns])
results_count.append(seurat_cca_df[cell_count.columns])
results_q05.append(seurat_cca_df[cell_count.columns])
results_count_q05.append(seurat_cca_df[cell_count.columns])
names.append('Seurat 3 CCA')

# add SPOTlight
results.append(spotlight_df[cell_count.columns])
results_count.append(spotlight_df[cell_count.columns])
results_q05.append(spotlight_df[cell_count.columns])
results_count_q05.append(spotlight_df[cell_count.columns])
names.append('SPOTlight')

# add NNLS
results.append(nnls_df[cell_count.columns])
results_count.append(nnls_df[cell_count.columns])
results_q05.append(nnls_df[cell_count.columns])
results_count_q05.append(nnls_df[cell_count.columns])
names.append('NNLS')

In [ ]:
for i, n in enumerate(names):
    if (type(n) is not str) and (type(n) is not type(None)):
        names[i] = n.__name__
            
names

In [ ]:
rcParams['figure.figsize'] = 20, 8
rcParams['font.size'] = 16
compare_precision_recall(pos_cell_count, results,
                         method_title=names, 
                         legend_loc=(1.1, 0.5))
plt.tight_layout()
plt.savefig(fig_path + '/ROC_comparison_all.pdf')

In [ ]:
rcParams['figure.figsize'] = 20, 8
rcParams['font.size'] = 16
compare_precision_recall(pos_cell_count, results_count,
                         method_title=names, 
                         legend_loc=(1.1, 0.5))
plt.tight_layout()
plt.savefig(fig_path + '/ROC_comparison_all_count.pdf')

In [ ]:
rcParams['figure.figsize'] = 20, 8
rcParams['font.size'] = 16
compare_precision_recall(pos_cell_count, results_q05,
                         method_title=names, 
                         legend_loc=(1.1, 0.5))
plt.tight_layout()
plt.savefig(fig_path + '/ROC_comparison_q05_all.pdf')

In [ ]:
# save combined object
results_dict = {}
for i, n in enumerate(names):
    results_dict[n] = results[i]
    
file = f'{results_folder}hyperparameters_v2/benchmark_objects.p'
pickle.dump(results_dict, file = open(file, "wb")) 

In [ ]:
rcParams['figure.figsize'] = 7,8
rcParams['font.size'] = 16

mod_names = ['LocationModelLinearDependentWMultiExperiment',
             'Stereoscope',
             'RCTD',
             'Seurat 3 PCA',
             'SPOTlight',
             'NNLS'
            ]

results_dict_sel = {}
for i, n in enumerate(mod_names):
    results_dict_sel[n] = results_dict[n]

    
if False:
    compare_precision_recall(pos_cell_count, list(results_dict_sel.values()),
                             method_title=list(results_dict_sel.keys()),
                             legend_loc=(-0.17, -0.6), curve='ROC')
    plt.tight_layout()
    plt.savefig(fig_path + '/ROC_comparison_selected_FigS2.pdf')
    plt.show()
    
compare_precision_recall(pos_cell_count, list(results_dict_sel.values()),
                         method_title=list(results_dict_sel.keys()),
                         legend_loc=(-0.17, -0.6))
plt.tight_layout()
plt.savefig(fig_path + '/ROC_comparison_selected_FigS2.pdf')

In [ ]:
rcParams['figure.figsize'] = 10,8
rcParams['font.size'] = 16

mod_names = ['LocationModelLinearDependentWMultiExperiment',
 'LocationModelLinearDependentWMultiExperimentNoMg',
 'LocationModelHierarchicalWMultiExperiment',
 'LocationModelMultiExperiment',
 'LocationModelLinearDependentWMultiExperimentLocationBackgroundNormGeneAlpha'
            ]

results_dict_sel = {}
for i, n in enumerate(mod_names):
    results_dict_sel[n] = results_dict[n]

if False:
    compare_precision_recall(pos_cell_count, list(results_dict_sel.values()),
                             method_title=list(results_dict_sel.keys()),
                             legend_loc=(-0.17, -0.6), curve='ROC')
    plt.tight_layout()
    plt.savefig(fig_path + '/ROC_comparison_simplified_FigS2.pdf')
    plt.show()

compare_precision_recall(pos_cell_count, list(results_dict_sel.values()),
                         method_title=list(results_dict_sel.keys()),
                         legend_loc=(-0.17, -0.6), curve='PR')
plt.tight_layout()
plt.savefig(fig_path + '/PR_comparison_simplified_FigS2.pdf')
plt.show()

In [ ]:
rcParams['figure.figsize'] = 10,8
rcParams['font.size'] = 16

mod_names = ['LocationModelLinearDependentWMultiExperiment',
 'LocationModelLinearDependentWMultiExperimentNoMg',
 'LocationModelMultiExperiment',
            ]

results_dict_sel = {}
for i, n in enumerate(mod_names):
    results_dict_sel[n] = results_dict[n]

if False:
    compare_precision_recall(pos_cell_count, list(results_dict_sel.values()),
                             method_title=list(results_dict_sel.keys()),
                             legend_loc=(-0.17, -0.6), curve='ROC')
    plt.tight_layout()
    plt.savefig(fig_path + '/ROC_comparison_simplified_paper_FigS2.pdf')
    plt.show()

compare_precision_recall(pos_cell_count, list(results_dict_sel.values()),
                         method_title=list(results_dict_sel.keys()),
                         legend_loc=(-0.17, -0.6), curve='PR')
plt.tight_layout()
plt.savefig(fig_path + '/PR_comparison_simplified_paper_FigS2.pdf')
plt.show()

#### Compares using RMSE on proportions

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = pd.DataFrame()
for i in range(len(results)):
    rmse[names[i]] = ((cell_proportions - results[i]) ** 2).mean(1)

rcParams['figure.figsize'] = 20, 5
sns.boxplot(data=rmse, orient="h", palette=sc.pl.palettes.default_102).set(
    xlabel='Root mean squared error'
)
plt.tight_layout()
plt.savefig(fig_path + '/RMSE_comparison_all.pdf')
plt.show()

In [ ]:
rcParams['figure.figsize'] = 12, 3
rcParams['pdf.fonttype'] = 42
rcParams['font.size'] = 16

mod_names = ['LocationModelLinearDependentWMultiExperiment',
             'Stereoscope',
             'RCTD',
             'SPOTlight',
             'Seurat 3 PCA',
             'NNLS']

from itertools import cycle
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])
my_palette = dict(zip(mod_names, colors))

sns.boxplot(data=rmse[mod_names], orient="h", palette=my_palette).set(
    xlabel='Root mean squared error'
)
plt.tight_layout()
plt.savefig(fig_path + '/RMSE_comparison_selected.pdf')
plt.show()

In [ ]:
rmse_q05 = pd.DataFrame()
for i in range(len(results)):
    rmse_q05[names[i]] = ((cell_proportions - results_q05[i]) ** 2).mean(1)
    
rcParams['figure.figsize'] = 15, 5
sns.boxplot(data=rmse_q05, orient="h", palette=sc.pl.palettes.default_102).set(
    xlabel='Root mean squared error'
)
plt.show()

## Plot ground truth and cell estimates in 2D

In [ ]:
mpl.rc_file_defaults()
mpl.rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
with mpl.rc_context({'figure.figsize': (5,5), 'axes.facecolor': "black"}):
    sc.pl.spatial(adata_vis_res, cmap='magma',
                  color=[f'cell_abundances_{i}' for i in cell_proportions.columns], ncols=5, 
                  size=0.015, 
                  img_key=None,
                  alpha_img=1, vmin=0, vmax='p99.2',
                  show=True, return_fig=False)

In [ ]:
mpl.rc_file_defaults()
mpl.rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
with mpl.rc_context({'figure.figsize': (5,5), 'axes.facecolor': "black"}):
    sc.pl.spatial(adata_vis_res, cmap='magma',
                  color=[f'q05_spot_factors{i}' for i in cell_proportions.columns], ncols=5, 
                  size=0.015, 
                  img_key=None,
                  alpha_img=1, vmin=0, vmax='p99.2',
                  show=True, return_fig=False)

In [ ]:
mpl.rc_file_defaults()
mpl.rcParams['pdf.fonttype'] = 42 # enables correct plotting of text

adata_vis_res.obs[[f'Stereoscope_{i}' for i in cell_proportions.columns]] \
= results_dict['Stereoscope']

with mpl.rc_context({'figure.figsize': (5,5), 'axes.facecolor': "black"}):
    sc.pl.spatial(adata_vis_res, cmap='magma',
                  color=[f'Stereoscope_{i}' for i in cell_proportions.columns], ncols=5, 
                  size=0.015, 
                  img_key=None,
                  alpha_img=1, vmin=0, vmax='p99.2',
                  show=True, return_fig=False)

In [ ]:
mpl.rc_file_defaults()
mpl.rcParams['pdf.fonttype'] = 42 # enables correct plotting of text

adata_vis_res.obs[[f'RCTD_{i}' for i in cell_proportions.columns]] \
= results_dict['RCTD']

with mpl.rc_context({'figure.figsize': (5,5), 'axes.facecolor': "black"}):
    sc.pl.spatial(adata_vis_res, cmap='magma',
                  color=[f'RCTD_{i}' for i in cell_proportions.columns], ncols=5, 
                  size=0.015, 
                  img_key=None,
                  alpha_img=1, vmin=0, vmax='p99.2',
                  show=True, return_fig=False)

In [ ]:
mpl.rc_file_defaults()
mpl.rcParams['pdf.fonttype'] = 42 # enables correct plotting of text

adata_vis_res.obs[[f'SPOTlight_{i}' for i in cell_proportions.columns]] \
= results_dict['SPOTlight']

with mpl.rc_context({'figure.figsize': (5,5), 'axes.facecolor': "black"}):
    sc.pl.spatial(adata_vis_res, cmap='magma',
                  color=[f'SPOTlight_{i}' for i in cell_proportions.columns], ncols=5, 
                  size=0.015, 
                  img_key=None,
                  alpha_img=1, vmin=0, vmax='p99.2',
                  show=True, return_fig=False)